# Conversation Buffer
---
En este notebook motraremos como usar `ConversationBufferMemory`. 

Este tipo de memoria permite el almacenamiento y la extraccion de los mensajes en una variable.

### Configuracion

La importacion la realizamos desde:

In [1]:
from langchain.memory import ConversationBufferMemory

Hagamos un ejemplo, crearemos una instancia y agregaremos informacion a la misma.

In [4]:
memory = ConversationBufferMemory()

# metodo para agregar informacion a la "memoria"
memory.save_context(
    inputs={"input": "hi"},
    outputs={"outputs": "what's up?"}
)

Podemos visualizar el contenido de la memoria usando el metodo `load_memory_variables`, dicho metodo devuelve lo que contiene la `memoria`, 

lo haremos de la siguiente forma:

In [5]:
memory.load_memory_variables({})

{'history': "Human: hi\nAI: what's up?"}

Ademas, podemos obtener el historial como una lista de mensajes (esto es util si lo estamos usando con un modelo de chat)

In [6]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    inputs={"input": "hi"},
    outputs={"output": "what's up?"}
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, example=False),
  AIMessage(content="what's up?", additional_kwargs={}, example=False)]}

**Nota:** 

La lista de mensajes la obtendremos como una lista de modelos de pydantic, tenemos 3 modelos 
- HumanMessage
- AIMessage
- SystemMessage

Recordemos que langchain esta basado en pydantic

## Uso en "chain"
---

Finalmente, veamos un ejemplo de como usarlo en un "chain".

Usaremos la configuracion verbose=True, que disponen los elementos de langchain para poder observar su funcionamiento en la consola, y asi podremos observar el "prompt".

In [7]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

# llm
llm = OpenAI(temperature=0)

# memory
memory = ConversationBufferMemory()

# chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)

Enviaremos un texto al chain, y observemos su funcionamiento.

In [8]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

Enviemos otro mensaje, para ver el prompt junto con el historial de la conversacion, hay que prestar atencion a la seccion de "Current conversation"

In [9]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

Enviemos otro mensaje para que se muestre en la conversacion.

In [10]:
conversation.predict(input="Tell me about yourself")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:  That's great! It's always nice to have a conversation with someone new. What would you like to talk about?
Human: Tell me about yourself
AI:

> Finished chain.


" Sure! I'm an AI created to help people with their everyday tasks. I'm programmed to understand natural language and provide helpful information. I'm also constantly learning and updating my knowledge base so I can provide more accurate and helpful answers."

Como podemos notar, se va "completando" la informacion en el prompt, a todo este texto le llamaremos el "contexto", que es toda la informacion con la cual el LLM procesa.

**Nota:** 

Debemos tener precaucion con el tamaño del "contexto", ya que cada modelo tiene una cantidad maxima de informacion que puede manejar, esta se mide en "tokens" y cuando se alcance el
limite del modelo, obtendremos un error.

En otros "notebooks" veremos posibles soluciones a este problema.